In [1]:
!pip uninstall -y numpy datasets

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: datasets 2.14.5
Uninstalling datasets-2.14.5:
  Successfully uninstalled datasets-2.14.5


In [2]:
!pip install --force-reinstall numpy==1.26.4 datasets==2.14.5

  Using cached numpy-1.26.4-cp39-cp39-win_amd64.whl.metadata (61 kB)
  Using cached datasets-2.14.5-py3-none-any.whl.metadata (19 kB)
  Using cached pyarrow-20.0.0-cp39-cp39-win_amd64.whl.metadata (3.4 kB)
  Using cached dill-0.3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached pandas-2.2.3-cp39-cp39-win_amd64.whl.metadata (19 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.5.0-cp39-cp39-win_amd64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.18-py39-none-any.whl.metadata (7.5 kB)
  Using cached fsspec-2023.6.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached huggingface_hub-0.32.0-py3-none-any.whl.metadata (14 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached PyYAML-6.0.2-cp39-cp39-win_amd64.whl.metadata (2.1 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.

In [ ]:
# Semantic Textual Similarity with BERT (STS-B)
# Working on Python9 environment
# Import libraries and setup
from transformers import AutoTokenizer, BertModel, TrainingArguments, Trainer
from datasets import load_dataset
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import os
# Disable wandb logging
os.environ["WANDB_DISABLED"] = "true"

c:\Users\ragha\anaconda3\envs\Python9\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Load STS-B dataset and tokenizer
dataset = load_dataset("glue", "stsb")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [5]:
# CELL 3: Tokenization function
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [6]:
# Define custom regression model class
class STSModel(nn.Module):
    def __init__(self):
        super(STSModel, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.regressor = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(768, 1),
            nn.Sigmoid()
        )

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.pooler_output
        logits = self.regressor(cls_output) * 5  # scale to [0, 5]
        return logits

model = STSModel()

In [7]:
#  Define metric function using Pearson correlation
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.squeeze()
    return {"pearson": pearsonr(predictions, labels)[0]}

In [8]:
import accelerate
print(accelerate.__version__)


1.7.0


In [9]:
# CELL 6: Define training arguments
training_args = TrainingArguments(
    output_dir="./sts_results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    do_eval=True,
    save_steps=500,
    eval_steps=500,
    report_to="none"
)

In [10]:
class STSTrainer(Trainer):
       def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
           labels = inputs.pop("labels")
           outputs = model(**inputs)
           loss = nn.MSELoss()(outputs.view(-1), labels.view(-1))
           return (loss, outputs) if return_outputs else loss

In [11]:
# CELL 7: Define custom trainer class for regression
trainer = STSTrainer(
       model=model,
       args=training_args,
       train_dataset=tokenized_dataset["train"],
       eval_dataset=tokenized_dataset["validation"],
       compute_metrics=compute_metrics,
   )


In [ ]:
# CELL 9: Train model
trainer.train()

Step,Training Loss
10,2.057500
20,2.059000
30,1.680400
40,1.592200
50,1.376000
60,1.242500
70,1.126600
80,1.083500
90,0.810500
100,0.946900


In [36]:
# CELL 10: Save model and tokenizer
model_path = "./bert_sts_model"
os.makedirs(model_path, exist_ok=True)
tokenizer.save_pretrained(model_path)
torch.save(model.state_dict(), f"{model_path}/model.pt")